# 체크용

In [ ]:
os.listdir('../../../cranberry2/Preprocessing/cache/lowess_filtered')

In [2]:
import pandas as pd
import os

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
print(len(df_match))

fs=os.listdir('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/')
len(fs)

10156


8804

In [15]:
df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

nonlist=[]
yeslist=[]
srate= 100
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        nonlist.append(df_match.loc[i,'path'])
    else:
        yeslist.append(df_match.loc[i,'path'])
        
len(nonlist),len(yeslist)

10156


(1352, 8804)

In [16]:
import pickle

cid = pickle.load(open('../../DL_model/caseids_diff','rb'))
cid = [e[:-3]+'vital' for e in cid]

existed = []
still=[]
for p in nonlist:
    if p in cid:
        existed.append(p)
    if p in yeslist:
        still.append(p)
len(cid),len(existed),len(still)

(591, 190, 1132)

# ECG 250Hz

In [7]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

srate= 250
cnt=0
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    
    if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'+filename):
        cnt +=1
        print(cnt,' : ', i, end=' ')
        #try: 
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_ecg_pleth/pacu_ecg_pleth/'+df_match.loc[i,'path']
        #print(rec_path)
        #print('1')
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        #print(vfile)
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx = max(0,start_idx)
                
                ext = df_vital[start_idx:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['Abs_time','ECG'])
                        extraction[fit:]=ext

                        print(extr)
                        print(df_match.loc[i,'path'])

                    else:
                        extraction = ext
                    #with open('vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        #pickle.dump(extraction, f)
                    print('...ready to save')
                else:
                    print('... ',df_match.loc[i,'NRS_time'],df_vital.loc[end_idx,'Abs_time'])
                    print(df_vital[0:end_idx])
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

10156
1  :  9 ...ready to save
2  :  23 ...ready to save
3  :  27 ...ready to save
4  :  37 ...ready to save
5  :  45 ...ready to save
6  :  47 ...ready to save
7  :  60 ...ready to save
8  :  70 ...ready to save
9  :  75 ...ready to save
10  :  81 ...ready to save
11  :  83 ...ready to save
12  :  84 ...ready to save
13  :  85 ...  2019-04-22 10:20:00 2019-04-22 10:19:59.008000
                      Abs_time  ECG
0      2019-04-22 10:12:27.880  NaN
1      2019-04-22 10:12:27.884  NaN
2      2019-04-22 10:12:27.888  NaN
3      2019-04-22 10:12:27.892  NaN
4      2019-04-22 10:12:27.896  NaN
...                        ...  ...
112777 2019-04-22 10:19:58.988  NaN
112778 2019-04-22 10:19:58.992  NaN
112779 2019-04-22 10:19:58.996  NaN
112780 2019-04-22 10:19:59.000  NaN
112781 2019-04-22 10:19:59.004  NaN

[112782 rows x 2 columns]
14  :  86 ...ready to save
15  :  87 ...  2019-04-24 02:25:00 2019-04-24 02:24:59.006000
                    Abs_time  ECG
0    2019-04-24 02:24:38.934  NaN
1 

102  :  482 ...ready to save
103  :  483 ...ready to save
104  :  489 ...ready to save
105  :  493 ...ready to save
106  :  495 ...ready to save
107  :  497 ...ready to save
108  :  503 ...ready to save
109  :  505 ...ready to save
110  :  508 ...ready to save
111  :  510 ...ready to save
112  :  512 ...ready to save
113  :  514 ...ready to save
114  :  515 ...ready to save
115  :  519 ...ready to save
116  :  521 ...ready to save
117  :  531 ...ready to save
118  :  533 ...ready to save
119  :  535 ...ready to save
120  :  557 ...ready to save
121  :  562 ...ready to save
122  :  565 ...ready to save
123  :  572 ...ready to save
124  :  579 ...ready to save
125  :  582 ...ready to save
126  :  591 ...ready to save
127  :  594 ...  2019-05-24 08:55:00 2019-05-24 08:54:59.005000
                     Abs_time  ECG
0     2019-05-24 08:52:45.653  NaN
1     2019-05-24 08:52:45.657  NaN
2     2019-05-24 08:52:45.661  NaN
3     2019-05-24 08:52:45.665  NaN
4     2019-05-24 08:52:45.669  NaN
.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-06f728372f01>", line 36, in <module>
    Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
  File "<ipython-input-7-06f728372f01>", line 36, in <listcomp>
    Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Tra

TypeError: object of type 'NoneType' has no len()